In [1]:
from preprocess import *

mediaitems = extract_metadata("/Users/samyuktha/Downloads/Pixtale KL Photos")

/Users/samyuktha/pixtale/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Processing img_6626.mov
Extracting metadata from img_6626.mov
img_6626.mov is a video file
Processing .ds_store
Extracting metadata from .ds_store
Processing img_6423.mov
Extracting metadata from img_6423.mov
img_6423.mov is a video file
Processing img_6488.jpeg
Extracting metadata from img_6488.jpeg
No GPS Data on  /Users/samyuktha/Downloads/Pixtale KL Photos/img_6488.jpeg
Processing img_6700.mov
Extracting metadata from img_6700.mov
img_6700.mov is a video file
Processing img_6588.jpeg
Extracting metadata from img_6588.jpeg
No GPS Data on  /Users/samyuktha/Downloads/Pixtale KL Photos/img_6588.jpeg


In [ ]:


import json

with open("mediaitems.json", "r") as json_file:
    mediaitems = json.load(json_file)


mediaitems = generate_descriptions(dir_path, mediaitems)




In [ ]:
mediaitems.sort(key=lambda item: item["datetime"])

mediaitems

In [ ]:
import json
from geminiai import *

with open("mediaitems.json", "r") as json_file:
    mediaitems = json.load(json_file)


narration_script = get_script(mediaitems)

narration_script

In [ ]:
import shutil
from moviepy.editor import (
    VideoFileClip,
    ImageClip,
    concatenate_videoclips,
    TextClip,
    CompositeVideoClip,
    concatenate_audioclips,
    AudioFileClip,
)
from moviepy.video.fx.resize import resize
import os
from moviepy.video import fx as vfx
from moviepy.video.fx import speedx


clips = []

# Define a standard resolution
standard_resolution = (1080, 1920)  # Width x Height


os.makedirs("./audio")

audio_clips = []
# Process each scene
for i in range(len(narration_script["scenes"])):
    scene = narration_script["scenes"][i]

    text = scene["text"]

    filepath = os.path.join(dir_path, scene["media_source"]).lower()

    print(filepath)

    if os.path.exists(filepath):

        if not os.path.exists("./audio/" + str(i) + ".mp3"):

            synthesize_text(text, "./audio/" + str(i))

        audio_clip = AudioFileClip("./audio/" + str(i) + ".mp3")

        duration = audio_clip.duration

        if filepath.endswith((".mp4", ".mov")):
            clip = VideoFileClip(filepath)

            if duration < clip.duration:

                clip = clip.subclip(0, duration)
                
            clip=clip.resize(newsize=(standard_resolution[0], standard_resolution[1]))

        else:
            clip = ImageClip(filepath).set_duration(duration)

        # txt_clip = (
        #     TextClip(
        #         scene["narration"]["text"],
        #         color="white",
        #         font="Arial",
        #         fontsize=50,
        #         method="caption",
        #         stroke_width=2,
        #     )
        #     .set_position("bottom")
        #     .set_duration(float(duration))
        # )
        # clip = CompositeVideoClip([clip, txt_clip])
        clips.append(clip)

        audio_clips.append(audio_clip)


# shutil.rmtree("./audio")
# Concatenate all clips
final_clip = concatenate_videoclips(clips, method="compose")

final_audio_clip = concatenate_audioclips(audio_clips)

final_audio_clip.write_audiofile("final_trip_audio.mp3")

if final_clip.duration != final_audio_clip.duration:

    if final_clip.duration > final_audio_clip.duration:
        # Speed up the video clip to match the audio clip's duration
        speed_factor = final_clip.duration / final_audio_clip.duration
        final_clip = final_clip.fx(vfx.speedx, speed_factor)
    else:
        # Speed up the audio clip to match the video clip's duration

        speed_factor = final_audio_clip.duration / final_clip.duration
        print(speed_factor)
        cmd = (
            'ffmpeg -y -i final_trip_audio.mp3 -filter:a "atempo={}" output.mp3'.format(
                speed_factor
            )
        )

        os.system(cmd)


# Write the result to a file
final_clip.write_videofile("final_trip_video.mp4")

In [ ]:
cmd = "ffmpeg -y -i final_trip_video.mp4 -i output.mp3 -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac combined_video.mp4"
os.system(cmd)

In [14]:
def list_voices():
    """Lists the available voices."""
    from google.cloud import texttospeech

    client = texttospeech.TextToSpeechClient()

    # Performs the list voices request
    voices = client.list_voices()

    english_voices = []

    for voice in voices.voices:

        for language_code in voice.language_codes:
            if "en" in language_code:

                english_voices.append(
                    {
                        "language_codes": voice.language_codes[0],
                        "ssml_gender": texttospeech.SsmlVoiceGender(
                            voice.ssml_gender
                        ).name,
                        "name": voice.name,
                        "voice": "{}-{}".format(
                            voice.name,
                            texttospeech.SsmlVoiceGender(voice.ssml_gender).name,
                        ),
                    }
                )

    return english_voices

print(list_voices())

[{'language_codes': 'en-AU', 'ssml_gender': 'FEMALE', 'name': 'en-AU-Neural2-A', 'voice': 'en-AU-Neural2-A-FEMALE'}, {'language_codes': 'en-AU', 'ssml_gender': 'MALE', 'name': 'en-AU-Neural2-B', 'voice': 'en-AU-Neural2-B-MALE'}, {'language_codes': 'en-AU', 'ssml_gender': 'FEMALE', 'name': 'en-AU-Neural2-C', 'voice': 'en-AU-Neural2-C-FEMALE'}, {'language_codes': 'en-AU', 'ssml_gender': 'MALE', 'name': 'en-AU-Neural2-D', 'voice': 'en-AU-Neural2-D-MALE'}, {'language_codes': 'en-AU', 'ssml_gender': 'FEMALE', 'name': 'en-AU-News-E', 'voice': 'en-AU-News-E-FEMALE'}, {'language_codes': 'en-AU', 'ssml_gender': 'FEMALE', 'name': 'en-AU-News-F', 'voice': 'en-AU-News-F-FEMALE'}, {'language_codes': 'en-AU', 'ssml_gender': 'MALE', 'name': 'en-AU-News-G', 'voice': 'en-AU-News-G-MALE'}, {'language_codes': 'en-AU', 'ssml_gender': 'MALE', 'name': 'en-AU-Polyglot-1', 'voice': 'en-AU-Polyglot-1-MALE'}, {'language_codes': 'en-AU', 'ssml_gender': 'FEMALE', 'name': 'en-AU-Standard-A', 'voice': 'en-AU-Standa

In [ ]:
from utils import *

public = "./static/public"
with open("data/narration_script.json", "r") as json_file:
    narration_script = json.load(json_file)

create_video(narration_script,public)

In [ ]:
import json

from utils import *


with open("data/mediaitems.json", "r") as file:
    mediaitems = json.load(file)

create_video(mediaitems, "./static/public")